**get model coordinates**

In [2]:
import plotly.express as px
import pandas as pd
import numpy as np

coor = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Coron/met_ model/distan_lat42.58lon-8.8046p3R4Km.csv")
px.set_mapbox_access_token("pk.eyJ1IjoiZ3JhbmFudHVpbiIsImEiOiJja3B4dGU4OTkwMTFmMm9ycnNhMjJvaGJqIn0.VWzx_PkD9A5cSUVsn_ijCA")
px.scatter_mapbox(coor, hover_data=['distance'],lat='lat', lon='lon',color='distance', title="nearest points",
                           color_continuous_scale=px.colors.cyclical.IceFire,)

**get station and sample sd speed mean hour before**

In [3]:
station = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Coron/station_data/sdspd_o.csv", usecols = ["time","sdspd_o"],parse_dates=["time"]).set_index("time")
print("raw data\n",station.head(24))

#show duplicated
df_duplicate = station[station.index.duplicated()]
print("duplicated")
print(df_duplicate)

#delete duplicated and resample
station = station[~station.index.duplicated(keep='first')]
station = station[station>0]
station = station.resample("H",label="right",closed="right").mean()

print("sd spd hour before")
print(station.head(4))
print("Variable statistics")
print(station.describe())

raw data
                       sdspd_o
time                         
2018-01-01 00:10:00  0.833334
2018-01-01 00:20:00  0.694445
2018-01-01 00:30:00  0.944445
2018-01-01 00:40:00  1.000001
2018-01-01 00:50:00  1.027779
2018-01-01 01:00:00  1.194445
2018-01-01 01:10:00  1.250001
2018-01-01 01:20:00  1.083334
2018-01-01 01:30:00  1.166668
2018-01-01 01:40:00  1.222223
2018-01-01 01:50:00  1.138890
2018-01-01 02:00:00  1.138890
2018-01-01 02:10:00  1.333334
2018-01-01 02:20:00  1.055556
2018-01-01 02:30:00  1.222223
2018-01-01 02:40:00  1.388890
2018-01-01 02:50:00  1.305557
2018-01-01 03:00:00  1.250001
2018-01-01 03:10:00  1.027779
2018-01-01 03:20:00  1.166668
2018-01-01 03:30:00  1.805557
2018-01-01 03:40:00  1.333334
2018-01-01 03:50:00  1.000001
2018-01-01 04:00:00  1.500001
duplicated
                      sdspd_o
time                         
2018-03-25 03:00:00  0.333334
2018-03-25 03:10:00  1.000001
2018-03-25 03:20:00  0.472223
2018-03-25 03:30:00  0.500000
2018-03-25 03:40:00

**get meteorologic model d0**

In [ ]:
#ria forecast d0
met_model = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Coron/met_ model/lat42.58lon-8.8046p3R4KmD0.csv",parse_dates=["time"]).drop(columns=["Unnamed: 0"]).set_index("time")
met_model.drop_duplicates().info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 42021 entries, 2018-01-01 01:00:00 to 2023-01-01 23:00:00
Data columns (total 75 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   dir0         42021 non-null  float64
 1   snow_prec0   42021 non-null  float64
 2   snowlevel0   42021 non-null  float64
 3   mod0         42021 non-null  float64
 4   wind_gust0   42021 non-null  float64
 5   mslp0        42021 non-null  float64
 6   temp0        42021 non-null  float64
 7   rh0          42021 non-null  float64
 8   visibility0  42021 non-null  float64
 9   lhflx0       42021 non-null  float64
 10  lwflx0       42021 non-null  float64
 11  conv_prec0   42021 non-null  float64
 12  prec0        42021 non-null  float64
 13  swflx0       42021 non-null  float64
 14  shflx0       42021 non-null  float64
 15  cape0        42021 non-null  float64
 16  cin0         42021 non-null  float64
 17  cfh0         42021 non-null  float64
 18  cfl0       

**Concatenate and dropnan**

In [ ]:
df_all = pd.concat([station,met_model],axis=1).dropna()
df_all.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 41321 entries, 2018-01-01 01:00:00 to 2022-12-30 23:00:00
Data columns (total 76 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   sdspd_o      41321 non-null  float64
 1   dir0         41321 non-null  float64
 2   snow_prec0   41321 non-null  float64
 3   snowlevel0   41321 non-null  float64
 4   mod0         41321 non-null  float64
 5   wind_gust0   41321 non-null  float64
 6   mslp0        41321 non-null  float64
 7   temp0        41321 non-null  float64
 8   rh0          41321 non-null  float64
 9   visibility0  41321 non-null  float64
 10  lhflx0       41321 non-null  float64
 11  lwflx0       41321 non-null  float64
 12  conv_prec0   41321 non-null  float64
 13  prec0        41321 non-null  float64
 14  swflx0       41321 non-null  float64
 15  shflx0       41321 non-null  float64
 16  cape0        41321 non-null  float64
 17  cin0         41321 non-null  float64
 18  cfh0       

**LGBMRegressor**

In [ ]:
from lightgbm.sklearn import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

#numpy randomseed
np.random.seed(1)

# target variable
Y = df_all.sdspd_o
X = df_all[df_all.columns[1:]]

MAE = []
for t in range (0,10):
  X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.1, )
  ml_model = LGBMRegressor(n_estimators=200).fit(X_train,y_train)
  y_pred = ml_model.predict(X_test)
  MAE.append(round(mean_absolute_error(y_test,y_pred),2))
  print("machine learning number:",t)
  df = pd.DataFrame({"y_test":y_test},index=y_test.index)
  df["y_pred"] = y_pred
  df["diff"] = df["y_pred"]-df["y_test"]
  print(df.describe())
print("\nDifferences last machine learning model")
fig = px.line(df["diff"].sort_index())
fig.show()
print("Mean absolute value all machine learning models")
df_r=pd.DataFrame({"MAE":MAE,});
df_r.describe()


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.063600 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 18221
[LightGBM] [Info] Number of data points in the train set: 37188, number of used features: 72
[LightGBM] [Info] Start training from score 0.765884
machine learning number: 0
            y_test       y_pred         diff
count  4133.000000  4133.000000  4133.000000
mean      0.766162     0.765982    -0.000180
std       0.519834     0.465099     0.228872
min       0.027778     0.231517    -1.321115
25%       0.388889     0.429374    -0.115860
50%       0.638889     0.640391     0.021243
75%       0.981482     0.931420     0.129724
max       3.888892     3.135392     1.390293
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.034877 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 18215
[LightGBM] [Info] Number of 

Mean absolute value all machine learning models


,MAE
count,10.000000
mean,0.170000
std,0.004714
min,0.160000
25%,0.170000
50%,0.170000
75%,0.170000
max,0.180000


**Shapiro test normality MAE samples tested**

In [ ]:
from scipy.stats import shapiro

sta, p = shapiro(MAE)
print("\np value:",p)
print("significance level alpha=0.01")

alpha = 0.01
if p>alpha:
  print("Accept null hypothesis: normal distribution variable MAE in the samples")
else:
  print("Reject null Hypothesis. Alternative hypothesis variable MAE no normal distribution in samples")


p value: 0.00027618114836513996
significance level alpha=0.01
Reject null Hypothesis. Alternative hypothesis variable MAE no normal distribution in samples


**Add time variables**

In [ ]:

df_all["hour"] = df_all.index.hour
df_all["month"] = df_all.index.month
df_all["dayofyear"] = df_all.index.dayofyear
df_all["weekofyear"] = df_all.index.isocalendar().week.astype(int)

#numpy randomseed
np.random.seed(1)

# target variable
Y = df_all.sdspd_o
X = df_all[df_all.columns[1:]]


MAE_t = []
for t in range (0,10):
  X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.1, )
  ml_model = LGBMRegressor(n_estimators=200).fit(X_train,y_train)
  y_pred = ml_model.predict(X_test)
  MAE_t.append(round(mean_absolute_error(y_test,y_pred),2))
  print("machine learning number:",t)
  df = pd.DataFrame({"y_test":y_test},index=y_test.index)
  df["y_pred"] = y_pred
  df["diff"] = df["y_pred"]-df["y_test"]
  print(df.describe())
print("\nDifferences last machine learning model")
fig = px.line(df["diff"].sort_index())
fig.show()
print("Mean absolute value all machine learning models")
df_r=pd.DataFrame({"MAE_t":MAE_t,});
df_r.describe()



[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.038316 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 18562
[LightGBM] [Info] Number of data points in the train set: 37188, number of used features: 76
[LightGBM] [Info] Start training from score 0.765884
machine learning number: 0
            y_test       y_pred         diff
count  4133.000000  4133.000000  4133.000000
mean      0.766162     0.764422    -0.001739
std       0.519834     0.464235     0.223212
min       0.027778     0.217704    -1.361038
25%       0.388889     0.430915    -0.113704
50%       0.638889     0.637251     0.019798
75%       0.981482     0.934441     0.124515
max       3.888892     3.164576     1.275420
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.035771 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 18555
[LightGBM] [Info] Number of 

Mean absolute value all machine learning models


,MAE_t
count,10.000000
mean,0.162000
std,0.004216
min,0.160000
25%,0.160000
50%,0.160000
75%,0.160000
max,0.170000


**T test two related samples. This is a test for the null hypothesis that two related samples have identical mean absolute error. Alternative hypothesis the  mean absolute error from the sample with time variables is less than the mean absolute error of from the sample with time variables. Significance level (alpha) = 0.01**

In [ ]:
from scipy import stats

alpha = 0.01
sta, p_value = stats.ttest_rel(MAE, MAE_t, alternative="greater")

print("\nMAE without time variables:", np.round(MAE,3))
print("\nMAE with time variables:", np.round(MAE_t,3))
print("\nalpha: ",alpha)
print("\np_value point: ",p_value)

if p_value < alpha:
    print("\nwe are rejecting null hypothesis  H0: MAE machine learning without time variables = MAE machine learning with time variables")
    print("\nwe are accepting alternative hypothesis: the MAE of the sample with time variables inputs is less than MAE of the sample without time variables inputs")
else:
    print("\nwe are accepting null hypothesis:  H0: MAE machine learning without time variables = MAE machine learning with time variables")


MAE without time variables: [0.17 0.17 0.17 0.17 0.17 0.17 0.16 0.17 0.17 0.18]

MAE with time variables: [0.16 0.16 0.16 0.16 0.16 0.16 0.16 0.16 0.17 0.17]

alpha:  0.01

p_value point:  0.0001012496610338204

we are rejecting null hypothesis  H0: MAE machine learning without time variables = MAE machine learning with time variables

we are accepting alternative hypothesis: the MAE of the sample with time variables inputs is less than MAE of the sample without time variables inputs


**Save algorithm**

In [ ]:
import pickle
from sklearn.pipeline import Pipeline


estimators = [('ml_model', ml_model)]
pipe = Pipeline(estimators)
print(pipe)
al_file={"x_var":X.columns,"coor":coor,"pipe":pipe}
pickle.dump(al_file, open("/content/drive/MyDrive/Colab Notebooks/Coron/algorithms/sdspd_coron_d0.al", 'wb'))

Pipeline(steps=[('ml_model', LGBMRegressor(n_estimators=200))])
